In [ ]:
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
driver = webdriver.Chrome("D:\\Installers\\chromedriver_win32\\chromedriver") # change path here
driver.get("https://www.youtube.com/results?search_query=%22email+management%22")

In [ ]:
user_data = driver.find_elements_by_xpath('//*[@id="video-title"]')
links = []
for i in user_data:
            links.append(i.get_attribute('href'))

print(len(links))

In [ ]:
df = pd.DataFrame(columns = ['link', 'title', 'description', 'views', 'category'])

In [ ]:
for link in links:
    if link is not None:
        driver.implicitly_wait(10)
        driver.get(link)

        id = link.strip('https://www.youtube.com/watch?v=')
        category = "Email Management"
        description = []

        title = driver.find_element_by_css_selector("h1.title yt-formatted-string").text
        description_elements = driver.find_elements_by_xpath("//*[@id='description']/yt-formatted-string/span")
        views = driver.find_element_by_xpath("//*[@id='count']/ytd-video-view-count-renderer/span[1]").text
        views = views.strip('views')

        if ',' in views:
            views = int(views.replace(',', ''))
        else:
            views = int(views)
        
        f = open("printlog.txt", "a", encoding='utf-8')
        f.write("Title - " + title + "\n")
        f.write("Description - ")

        for elem in description_elements:
            description.append(elem.text)
            f.write(elem.text + "\n")
        
        f.write("Views - " + str(views) + "\n")
        f.close()

        df.loc[len(df)] = [id, title, description, views, category]